In [1]:
from transformers import pipeline
from rouge_score import rouge_scorer

classifier = pipeline(
    "text-classification", 
    model="AbdullahBarayan/ModernBERT-base-doc_en-Cefr",
    device=0,
    top_k=None,
)

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

2026-01-25 17:06:08.005167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769360768.023601   14748 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769360768.029638   14748 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769360768.047055   14748 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769360768.047072   14748 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769360768.047074   14748 computation_placer.cc:177] computation placer alr

In [2]:
import json
from tqdm import tqdm

cefr_labels = ["A1", "A2", "B1", "B2", "C1", "C2"]
prediction_file = "billsum_v4_1/predictions/oob/ca_test_data_final_OFFICIAL.jsonl"

cefr_scores = {label: [] for label in cefr_labels}
rouge_scores = {label: [] for label in cefr_labels}

dataset = [line.strip() for line in open(prediction_file)]

for i in tqdm(range(len(dataset))):
    
    pred_json = json.loads(dataset[i])

    predictions = pred_json["predictions"]
    texts = [predictions[label] for label in cefr_labels]
    
    cefr_probs = classifier(texts, batch_size=6)

    for label, single_probs in zip(cefr_labels, cefr_probs):
        single_probs = {p["label"]: p["score"] for p in single_probs}
        cefr_scores[label].append(single_probs[label])
        
        single_scores = scorer.score(pred_json["summary"], predictions[label])
        rouge_scores[label].append(single_scores["rougeL"])
        

100%|██████████| 100/100 [00:32<00:00,  3.04it/s]


In [3]:
for k, scores in cefr_scores.items():
    cefr = sum(scores) * 100 / len(scores)
    rscores = rouge_scores[k]
    rouge = sum([score.fmeasure for score in rscores]) * 100 / len(rscores)
    print(k, cefr, rouge)


A1 0.6710009019075187 14.90231575756871
A2 4.295626783480088 15.994375851707503
B1 15.458475010909751 17.723811897507034
B2 40.16529539524345 17.88314642419055
C1 74.71102166501805 17.563971124453726
C2 0.14133713910996448 17.55408841775036
